In [5]:
import numpy as np
import yaml
from inferenceGUI import inferenceGUI

# Config

In [6]:
with open('./config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
acc_max = config['acc_max']
gyro_max = config['gyro_max']
confidence = config['confidence']

In [7]:
# load data from ./saved_data
X_test = np.load('./saved_data/X_test.npy')
y_test = np.load('./saved_data/y_test.npy')
# shuffle the X_test and y_test in the same order
ran_index = np.random.permutation(len(X_test))
X_test = X_test[ran_index]
y_test = y_test[ran_index]
test_len = len(X_test)
y_pred = np.random.randint(2, size=test_len)
print(X_test.shape)
print(y_test.shape)

(338, 50, 6)
(338,)


In [8]:
inferenceGUI = inferenceGUI(X_test, y_test, 
                             acc_max, gyro_max,
                             port='/dev/tty.usbmodem103', baudrate=115200, MCU = 'STM32', confidence = confidence)

# inferenceGUI = inferenceGUI(X_test, y_test, 
#                              acc_max, gyro_max,
#                              port='/dev/tty.usbmodem102', baudrate=115200, MCU = 'MAX', confidence = confidence)

inferenceGUI.launch()

SerialException: could not open port '/dev/tty.usbmodem103': FileNotFoundError(2, '系统找不到指定的路径。', None, 3)